# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Bhattacharyya  ->  M. Bhattacharyya  |  ['M. Bhattacharyya']
N. Storm  ->  N. Storm  |  ['N. Storm']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 51 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.15028


extracting tarball to tmp_2510.15028...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.15723
extracting tarball to tmp_2510.15723... done.


N. Storm  ->  N. Storm  |  ['N. Storm']


Found 130 bibliographic references in tmp_2510.15723/aa57188-25.bbl.
Issues with the citations
syntax error in line 53: '=' expected
Retrieving document from  https://arxiv.org/e-print/2510.15816
extracting tarball to tmp_2510.15816...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.15723-b31b1b.svg)](https://arxiv.org/abs/2510.15723) | **Rare Find: Discovery and chemo-dynamical properties of two s-process enhanced RR Lyrae stars**  |
|| V. D'Orazi, et al. -- incl., <mark>N. Storm</mark> |
|*Appeared on*| *2025-10-20*|
|*Comments*| *Accepted for publication in A&A. 16 pages, 11 figures. Abstract shortened*|
|**Abstract**|            We report the serendipitous discovery of two RR Lyrae stars exhibiting significant s-process element enrichment, a rare class previously represented solely by TY Gruis. Our goal is to characterise these objects chemically and dynamically, exploring their origins and evolutionary histories. Using high-resolution spectroscopy from HERMES@AAT and UVES@VLT, we derived detailed chemical abundances of key s-process elements, carbon along with $\alpha$-elements. We also employed Gaia DR3 astrometric data to analyse their kinematics, orbital properties, and classify their Galactic population membership. We compared observational results with theoretical asymptotic giant branch nucleosynthesis models to interpret their enrichment patterns. Both stars exhibit clear signatures of s-process enrichment, with significant overabundances in second-peak elements such as Ba and La compared to first-peak Y and Zr. Comparison with AGB nucleosynthesis models suggests their progenitors experienced pollution of s-process-rich material, consistent with early binary interactions. However, notable discrepancies in dilution factors highlight the need for more refined low-metallicity asymptotic giant branch (AGB) models. We also explore and discuss alternative scenarios, including sub-luminous post-AGB-like evolution or double episodes of mass this http URL findings confirm the existence of s-process-enhanced RR Lyrae stars and demonstrate the importance of combining chemical and dynamical diagnostics to unveil their complex evolutionary pathways. Future detailed binary evolution modelling and long-term orbital monitoring are essential to resolve their formation scenarios and assess the role of binarity in the evolution of pulsating variables.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.15028-b31b1b.svg)](https://arxiv.org/abs/2510.15028) | **Impact of Neutrino Flavor Conversions on Neutron Star Merger Dynamics, Ejecta, Nucleosynthesis, and Multi-Messenger Signals**  |
|| Y. Qiu, et al. -- incl., <mark>M. Bhattacharyya</mark> |
|*Appeared on*| *2025-10-20*|
|*Comments*| *19 pages, 13 figures, will be submitted to Phys.Rev.D*|
|**Abstract**|            We present numerical relativity simulations of binary neutron star mergers incorporating neutrino flavor transformations triggered by fast flavor instability, quantum many-body effects, or potential beyond standard model physics. In both long-lived and short-lived remnant scenarios, neutrino flavor conversions modify species-dependent neutrino luminosities and mean energies, and drive the matter towards more neutron rich conditions. They produce up to $300\%$ more neutron rich ejecta and significantly boost the r-process yields, especially in low-density, near-equatorial outflows. We identify regions unstable to fast flavor instabilities and find that these instabilities persist despite flavor conversions. We further test the sensitivity to the equilibration timescale of the flavor conversions, finding that slower flavor conversions can interact with thermodynamic equilibration, and increase the neutron richness of the ejecta. Flavor conversions may also contribute to stronger gravitational wave and neutrino emissions, pointing to a correlation between neutrino transport and merger dynamics. These results highlight the potential impact of flavor conversions while motivating future work to improve on theoretical understanding of flavor instabilities in global simulations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.15816-b31b1b.svg)](https://arxiv.org/abs/2510.15816) | **BREAKFAST: A Framework for general joint BA duty and follow-up guidance of multiple $γ$-ray monitors**  |
|| C.-W. Wang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-20*|
|*Comments*| **|
|**Abstract**|            With the growing number of gamma-ray monitors in operation, several research teams have adopted a strategy of joint operation and scientific duty to improve efficiency. A successful example is the GECAM-HXMT-SVOM (GHS) constellation collaboration, which sets a precedent for other gamma-ray monitor constellations. However, joint duty also presents challenges to Burst Advocates (BAs), including the increased number of triggers and, more importantly, the frequent switching between various systems due to incompatibilities among different missions, which complicates the situation. To address the current requirements of multi-wavelength and multi-messenger astronomy, we developed a customized framework for unified trigger processing within the GHS joint duty, named "BA's Rapid Evaluation and Analysis Kit for Formulating Alerts and Summary Tools" (BREAKFAST). This framework incorporates a series of automated, semi-automated, and manual pipelines designed to rapidly process triggers of prompt emissions in the gamma-ray band from different instruments, while maintaining flexible compatibility for future missions. The pursuit of BREAKFAST goes beyond merely providing trigger processing for BAs. BREAKFAST also aims to filtering high-value targets and guiding follow-up telescopes through rapid analysis and reporting, thus serving as an important bridge between prompt emission observations and afterglow observations. To this end, a suite of comprehensive analysis modules is included in BREAKFAST, particularly the specially designed module that predicts X-ray afterglow brightness based on prompt emission properties. The framework's effectiveness has already been demonstrated in recent observational campaigns, and it is expected to play a significant role in the discovery and observation of peculiar transients in the future.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.15723/./figures/CeY_FeH.jpeg', 'tmp_2510.15723/./figures/orbit.png', 'tmp_2510.15723/./figures/bensby_comp.png']
copying  tmp_2510.15723/./figures/CeY_FeH.jpeg to _build/html/
copying  tmp_2510.15723/./figures/orbit.png to _build/html/
copying  tmp_2510.15723/./figures/bensby_comp.png to _build/html/
exported in  _build/html/2510.15723.md
    + _build/html/tmp_2510.15723/./figures/CeY_FeH.jpeg
    + _build/html/tmp_2510.15723/./figures/orbit.png
    + _build/html/tmp_2510.15723/./figures/bensby_comp.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\vmic}{V_{\mathrm{mic}}}$
$\newcommand{\msun}{M_\odot}$</div>



<div id="title">

# Rare Find: Discovery and chemo-dynamical properties of two s-process enhanced RR Lyrae stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.15723-b31b1b.svg)](https://arxiv.org/abs/2510.15723)<mark>Appeared on: 2025-10-20</mark> -  _Accepted for publication in A&A. 16 pages, 11 figures. Abstract shortened_

</div>
<div id="authors">

V. D'Orazi, et al. -- incl., <mark>N. Storm</mark>

</div>
<div id="abstract">

**Abstract:**            We report the serendipitous discovery of two RR Lyrae stars exhibiting significant s-process element enrichment, a rare class previously represented solely by TY Gruis. Our goal is to characterise these objects chemically and dynamically, exploring their origins and evolutionary histories. Using high-resolution spectroscopy from HERMES@AAT and UVES@VLT, we derived detailed chemical abundances of key s-process elements, carbon along with $\alpha$-elements. We also employed Gaia DR3 astrometric data to analyse their kinematics, orbital properties, and classify their Galactic population membership. We compared observational results with theoretical asymptotic giant branch nucleosynthesis models to interpret their enrichment patterns. Both stars exhibit clear signatures of s-process enrichment, with significant overabundances in second-peak elements such as Ba and La compared to first-peak Y and Zr. Comparison with AGB nucleosynthesis models suggests their progenitors experienced pollution of s-process-rich material, consistent with early binary interactions. However, notable discrepancies in dilution factors highlight the need for more refined low-metallicity asymptotic giant branch (AGB) models. We also explore and discuss alternative scenarios, including sub-luminous post-AGB-like evolution or double episodes of mass this http URL findings confirm the existence of s-process-enhanced RR Lyrae stars and demonstrate the importance of combining chemical and dynamical diagnostics to unveil their complex evolutionary pathways. Future detailed binary evolution modelling and long-term orbital monitoring are essential to resolve their formation scenarios and assess the role of binarity in the evolution of pulsating variables.         

</div>

<div id="div_fig1">

<img src="tmp_2510.15723/./figures/CeY_FeH.jpeg" alt="Fig2" width="100%"/>

**Figure 2. -** [Ce/Y] ratios as a function of metallicity [Fe/H]. (*fig:cey_feh*)

</div>
<div id="div_fig2">

<img src="tmp_2510.15723/./figures/orbit.png" alt="Fig3" width="100%"/>

**Figure 3. -** Integrated orbits of DQ Hya (thick solid line: distance from parallax inversion; dashed line: distance estimated from the RR Lyrae magnitude–metallicity relation) and ASAS J153830–6906.4 (thin solid line: distance from parallax inversion), computed using the Milky Way potential models by \cite{mcmillan2017}(M17, left panel) and \cite{allen1991}(AS91, right panel). (*fig:orbit*)

</div>
<div id="div_fig3">

<img src="tmp_2510.15723/./figures/bensby_comp.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison of the orbital properties, eccentricity and maximum distance from the Galactic plane, of DQ Hya and ASAS J153830–6906.4 with the sample of disc stars from \cite{bensby2014}(B14).
    The contours represent the 68\%, 95\%, and 99.7\% confidence intervals from $10^5$ orbit realisations; line styles and colours follow those used in Figure \ref{fig:orbit}.
    Circle and square markers indicate stars classified by B14 as likely thin disc and thick disc members, respectively, based on a thick-to-thin disc likelihood ratio of $<0.5$ and $>2$. Star-shaped markers represent stars in between the two components with intermediate likelihood ratios.
    Both our orbit integrations and those in B14 adopt the Galactic potential from \cite{allen1991}. (*fig:bcomp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.15723"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

117  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
